In [17]:
import numpy as np
from sklearn.decomposition import PCA

در اینجا برای تعریف تابع اینفورمیشن گین، برای سادگی کار در ادامه، از تعریف دو گرهی که هر ویژگی داده ها را تقسیم بندی میکند استفاده میکنیم و تابع اینوفرمیشن گینز را برای ویژگی های مختلف تعریف میکنیم که شامل یک دیکشنری است که به ازای هر ویژگی که در اینجا با توجه به استفاده از آنالیز مولفه های اصلی 10 تاست، تعریف میشود و به ازای هر ویژگی در این دیکشنری یک بهره اطلاعات ذخیره میشود. . ، 

In [18]:
def entropy(y):
    values, counts = np.unique(y, return_counts=True)
    probabilites = counts / len(y)
    entropy = -np.sum(probabilites * np.log2(probabilites))
    return entropy

def information_gain(y, y_left_node, y_right_node):
    p_left = len(y_left_node) / len(y)
    p_right = len(y_right_node) / len(y)
    info_gain = entropy(y) - (p_left * entropy(y_left_node) + p_right * entropy(y_right_node))
    return info_gain

def information_gains(x, y):
    information_gains = {}  
    for feature in range(10):
        information_gains[feature] = information_gain(y,y_left_node,y_right_node)
    return information_gains

In [19]:
class Node:
    def __init__(self, feature=None, threshold=None, choice=None, left_node=None, right_node=None):
        self.feature = feature
        self.threshold = threshold
        self.choice = choice
        self.left_node = left_node
        self.right_node = right_node
        
        
def fit(X, y, max_depth=None):
    #در ابتدا شرطی قرار میدهیم که با توجه به حداکثر عمقی که تعریف میکنیم این الگوریتم ادامه پیدا کند . 
    #شرط بعدی این است که در یک داده ی جداسازی شده اگر تمامی برچسب ها متعلق به یک کلاس بودند، این الگوریتم متوفق میشود. 
    # با توقف این الگوریتم، شماره ی کلاسی که بیشترین تکرار را دارد در متغیر(choice) ذخیره میشود.   
    if max_depth is not None and max_depth == 0 or len(np.unique(y)) == 1:
        choice = np.argmax(np.bincount(y))
        return Node(choice=choice)
    # این متغیرها برای انتخاب بهترین بهره ی اطلاعاتی و بهترین ویژگی و آستانه ی مناسب برای داده ها مقداردهی اولیه میشوند..
    best_info_gain = -np.inf
    best_feature = None
    best_threshold = None
    
    # در این مرحله، از بین 10 ویژگی که در میان دادگان وجود دارد و بین آستانه های مختلف، یک آستانه و ویژگی بهینه برای تقسیم داده ها به دو گره مختلف انتخاب میشوند.
    # با بررسی مقدار داده ها بعد از پی سی ای، به این نتیجه میرسیم که داده ها بین -5 و 5 قرار میگیرند و با سرچ بین آستانه های مختلف، میتوانیم آستانه ی بهینه را انتخاب کنیم.
    # دلیل وجود این اعداد این است که اگر به صورت پیوسته آستانه سرچ میشد، بار محاسباتی بالایی داشت، اما با انتخاب گسسته بین اعداد صحیح، دقت تفاوت زیادی نمیکند و بار محاسباتی به شدت کاهش می یابد.
    
    for feature in range(10):
        thresholds = [-4,-3,-2,-1,0,1,2,3,4]
        for threshold in thresholds:
            #  در اینجا به ازای ویژگی ها و آستانه های مختلف، مقادیر داده با آستانه مقایسه میشوندو با توجه به درستی بزرگ یا کوچک بودن این عبارت، برچسب های گره های چپ و راست تشکیل میشوند. 
            indice = X[:, feature] <= threshold
            y_left_node = y[indice]
            y_right_node = y[~indice]
            inform_gain = information_gain(y, y_left_node, y_right_node)
            # در اینجا نیز عملیات مقایسه بین ویژگی های مختلف و آستانه های مختلف برای هر داده برای تشکیل گره با توجه به بهره ی اطلاعاتی انجام میشود.
            if inform_gain > best_info_gain:
                best_info_gain = inform_gain
                best_feature = feature
                best_threshold = threshold
    # در اینجا هم بعد از تمام شدن حلقه، مقادیر دادگان و برچسب های آنها باهم دیگر آپدیت میشوند. 
    indice = X[:, best_feature] <= best_threshold
    X_left_node, y_left_node = X[indice], y[indice]
    X_right_node, y_right_node = X[~indice], y[~indice]
    # در اینجا درخت به صورت بازگشتی آموزش میبیند تا زمانی که عمق تعیین شده ی ما به صفر برسد. 
    left_node = fit(X_left_node, y_left_node, max_depth - 1) if max_depth is not None else fit(X_left_node, y_left_node)
    right_node = fit(X_right_node, y_right_node, max_depth - 1) if max_depth is not None else fit(X_right_node, y_right_node)
    
    return Node(feature=best_feature, threshold=best_threshold, left_node=left_node, right_node=right_node)

در اینجا دادگان را لود میکنیم و ابتدا به 0 و1 نرمالیزه میکنیم. سپس به آرایه با سایز مورد نظر تغییر سایز میدهیم و بعد بر روی داده ی تست و آموزشی، آنالیز مولفه اصلی با 10 مولفه اعمال میکنیم و در نهایت مدل را با تعیین یک عمق حداکثری آموزش می دهیم. 

In [20]:
#x_train = np.load('F:/mnist/x_train.npy')
#y_train = np.load('F:/mnist/y_train.npy')
#x_test = np.load('F:/mnist/x_test.npy')
#y_test = np.load('F:/mnist/y_test.npy')
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [21]:
x_train = (x_train - np.min(x_train)) / (np.max(x_train) - np.min(x_train))
x_test = (x_test - np.min(x_test)) / (np.max(x_test) - np.min(x_test))

In [22]:
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)

In [23]:
pca = PCA(n_components=10)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

In [25]:
max_depth = 10
model = fit(x_train, y_train, max_depth)

در اینجا برای مرحله ی تست، با توجه به ورودی که به تابع پیش بینی میدهیم، بررسی میشود که اگر با توجه به مدل فیت شده و ورودی تست داده شده، اگر برای متغیر انتخاب، ما ورودی داشتیم، متغیر انتخاب به عنوان گره نهایی که داده را طبقه بندی کرده است و پیش بینی نهایی ماست، بازگردانده میشود، در غیر انیصورت باز هم با مقایسه ی با آستانه، به گره های چپ و راست برده میشوند تا پیش بینی به درستی انجام شود.
در نهایت هم به تعداد دادگان تست، در لیست پیشبینی، تخمین های مربوط به دادگان تست ذخیره میشوند و در مرحله ی آخر برای محاسبه ی دقت الگوریتم، این لیست پیشبینی با برچسب های دادگان اصلی مقایسه خواهد شد. 

In [28]:

def predict(model, X):
    if model.choice is not None:
        return model.choice
    if X[model.feature] <= model.threshold:
        return predict(model.left_node, X)
    else:
        return predict(model.right_node, X)
predictions = []
for i in range(len(x_test)):
    prediction = predict(model, x_test[i])
    predictions.append(prediction)

accuracy = np.sum(predictions == y_test) / len(y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8093
